import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=1751
cls_num=4
btch=80
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=EthanolLevel0(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (504, 1752)
first_test_shape= (500, 1752)
classes_quantity= 4
tr_lbls=	 {1, 2, 3, 4}
Count_labels= [126 126 126 126]
max(train_feature_Altitude)= 4.0
min(train_feature_Altitude)= -0.99315624
first_train_sample=
 [ 1.         -0.94066444 -0.94086231 ... -0.9264013  -0.92721665
 -0.92742172]
1 126	2 126	3 126	4 126	

In [3]:
np.shape(ecg)

(504, 1752)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 126	2 126	3 126	4 126	
 max = 126


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (126, 1752)
Up to class  2 train shape =  (252, 1752)
Up to class  3 train shape =  (378, 1752)
Up to class  4 train shape =  (504, 1752)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[  0 126 126 126 126]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  2.0126435999999996
min magnitude class 1  =  -0.9870482
after normalizing >>
max magnitude class 1  =  0.9591200363364702
min magnitude class 1  =  -0.996018902061441

 cls 2  >> 
max magnitude class 2  =  2.0131951000000003
min magnitude class 2  =  -0.9829150500000007
after normalizing >>
max magnitude class 2  =  0.9594794929728245
min magnitude class 2  =  -0.9933249978065979

 cls 3  >> 
max magnitude class 3  =  2.0126618000000005
min magnitude class 3  =  -0.9908758300000002
after normalizing >>
max magnitude class 3  =  0.9591318987313604
min magnitude class 3  =  -0.9985136745093239

 cls 4  >> 
max magnitude class 4  =  2.0080999
min magnitude class 4  =  -0.9925059200000004
after normalizing >>
max magnitude class 4  =  0.9561585438276738
min magnitude class 4  =  -0.999576134470075
each class and its windows =
1 126	2 124	3 126	4 124	
 max instance in classes of test windows = 126


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.999576134470075
max tst = 0.9594794929728245


In [14]:
len(wndws_test)

500

In [15]:
np.shape(wndws_test)

(500, 1752)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(126, 1752)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (504, 1751)
y_train =>  (504,)
X_test  =>  (500, 1751)
y_test  =>  (500,)
X_valid  =>  (504, 1751)
y_valid  =>  (504,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
7/7 [==============================] - 5s 77ms/step - loss: 1.6234 - accuracy: 0.1925 - val_loss: 1.4313 - val_accuracy: 0.2500
Epoch 2/1000
7/7 [==============================] - 0s 27ms/step - loss: 1.4292 - accuracy: 0.2579 - val_loss: 1.5192 - val_accuracy: 0.2500
Epoch 3/1000
7/7 [==============================] - 0s 28ms/step - loss: 1.4314 - accuracy: 0.2401 - val_loss: 1.4321 - val_accuracy: 0.2500
Epoch 4/1000
7/7 [==============================] - 0s 28ms/step - loss: 1.4115 - accuracy: 0.2242 - val_loss: 1.5156 - val_accuracy: 0.2500
Epoch 5/1000
7/7 [==============================] - 0s 31ms/step - loss: 1.4406 - accuracy: 0.2242 - val_loss: 1.3971 - val_accuracy: 0.2837
Epoch 6/1000
7/7 [==============================] - 0s 28ms/step - loss: 1.3918 - accuracy: 0.2698 - val_loss: 1.3878 - val_accuracy: 0.2500
Epoch 7/1000
7/7 [==============================] - 0s 27ms/step - loss: 1.3967 - accuracy: 0.2639 - val_loss: 1.4164 - val_accuracy: 0.2540
Epoch 8/1000


Epoch 59/1000
7/7 [==============================] - 0s 30ms/step - loss: 0.9571 - accuracy: 0.6012 - val_loss: 0.9604 - val_accuracy: 0.6111
Epoch 60/1000
7/7 [==============================] - 0s 27ms/step - loss: 0.9509 - accuracy: 0.5774 - val_loss: 0.9855 - val_accuracy: 0.5675
Epoch 61/1000
7/7 [==============================] - 0s 30ms/step - loss: 1.0148 - accuracy: 0.5714 - val_loss: 0.9082 - val_accuracy: 0.6151
Epoch 62/1000
7/7 [==============================] - 0s 27ms/step - loss: 0.9054 - accuracy: 0.6210 - val_loss: 0.9275 - val_accuracy: 0.5893
Epoch 63/1000
7/7 [==============================] - 0s 27ms/step - loss: 0.9030 - accuracy: 0.6091 - val_loss: 0.9658 - val_accuracy: 0.5615
Epoch 64/1000
7/7 [==============================] - 0s 28ms/step - loss: 0.9178 - accuracy: 0.6210 - val_loss: 0.8952 - val_accuracy: 0.6111
Epoch 65/1000
7/7 [==============================] - 0s 28ms/step - loss: 0.8453 - accuracy: 0.6389 - val_loss: 0.9709 - val_accuracy: 0.5595
Epoch 

7/7 [==============================] - 0s 27ms/step - loss: 0.2875 - accuracy: 0.8869 - val_loss: 0.1267 - val_accuracy: 0.9623
Epoch 117/1000
7/7 [==============================] - 0s 27ms/step - loss: 0.1310 - accuracy: 0.9544 - val_loss: 0.0914 - val_accuracy: 0.9762
Epoch 118/1000
7/7 [==============================] - 0s 27ms/step - loss: 0.6761 - accuracy: 0.8135 - val_loss: 0.2727 - val_accuracy: 0.9107
Epoch 119/1000
7/7 [==============================] - 0s 28ms/step - loss: 0.1814 - accuracy: 0.9524 - val_loss: 0.1368 - val_accuracy: 0.9603
Epoch 120/1000
7/7 [==============================] - 0s 30ms/step - loss: 0.1850 - accuracy: 0.9405 - val_loss: 0.0670 - val_accuracy: 0.9940
Epoch 121/1000
7/7 [==============================] - 0s 27ms/step - loss: 0.0758 - accuracy: 0.9802 - val_loss: 0.1469 - val_accuracy: 0.9325
Epoch 122/1000
7/7 [==============================] - 0s 27ms/step - loss: 0.2821 - accuracy: 0.8869 - val_loss: 0.3300 - val_accuracy: 0.8730
Epoch 123/1000

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

16/16 [==============================] - 0s 7ms/step
array([[55, 24, 30, 17],
       [46, 28, 30, 20],
       [23, 31, 28, 44],
       [18, 14, 34, 58]], dtype=int64)
              precision    recall  f1-score   support

           2       0.39      0.44      0.41       126
           3       0.29      0.23      0.25       124
           4       0.23      0.22      0.23       126
           5       0.42      0.47      0.44       124

    accuracy                           0.34       500
   macro avg       0.33      0.34      0.33       500
weighted avg       0.33      0.34      0.33       500



In [27]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
7/7 [==============================] - 1s 95ms/step - loss: 1.9258 - accuracy: 0.2262 - val_loss: 1.4169 - val_accuracy: 0.2500
Epoch 2/1000
7/7 [==============================] - 0s 48ms/step - loss: 1.4474 - accuracy: 0.2480 - val_loss: 1.4645 - val_accuracy: 0.2500
Epoch 3/1000
7/7 [==============================] - 0s 47ms/step - loss: 1.4431 - accuracy: 0.2381 - val_loss: 1.4077 - val_accuracy: 0.2500
Epoch 4/1000
7/7 [==============================] - 0s 48ms/step - loss: 1.4015 - accuracy: 0.2579 - val_loss: 1.4349 - val_accuracy: 0.2500
Epoch 5/1000
7/7 [==============================] - 0s 47ms/step - loss: 1.4127 - accuracy: 0.2381 - val_loss: 1.4233 - val_accuracy: 0.2500
Epoch 6/1000
7/7 [==============================] - 0s 47ms/step - loss: 1.4143 - accuracy: 0.2183 - val_loss: 1.4080 - val_accuracy: 0.2500
Epoch 7/1000
7/7 [==============================] - 0s 47ms/step - loss: 1.4058 - accuracy: 0.2302 - val_loss: 1.3924 - val_accuracy: 0.2500
Epoch 8/1000


Epoch 59/1000
7/7 [==============================] - 0s 47ms/step - loss: 1.0353 - accuracy: 0.5456 - val_loss: 1.0350 - val_accuracy: 0.5377
Epoch 60/1000
7/7 [==============================] - 0s 47ms/step - loss: 1.0024 - accuracy: 0.5635 - val_loss: 1.0818 - val_accuracy: 0.5238
Epoch 61/1000
7/7 [==============================] - 0s 47ms/step - loss: 1.0091 - accuracy: 0.5536 - val_loss: 1.0506 - val_accuracy: 0.5238
Epoch 62/1000
7/7 [==============================] - 0s 47ms/step - loss: 1.0299 - accuracy: 0.5516 - val_loss: 1.0758 - val_accuracy: 0.5218
Epoch 63/1000
7/7 [==============================] - 0s 48ms/step - loss: 0.9694 - accuracy: 0.5813 - val_loss: 1.0488 - val_accuracy: 0.5357
Epoch 64/1000
7/7 [==============================] - 0s 53ms/step - loss: 0.9771 - accuracy: 0.5774 - val_loss: 0.9550 - val_accuracy: 0.5893
Epoch 65/1000
7/7 [==============================] - 0s 47ms/step - loss: 0.9284 - accuracy: 0.6032 - val_loss: 1.0383 - val_accuracy: 0.5536
Epoch 

7/7 [==============================] - 0s 47ms/step - loss: 0.2250 - accuracy: 0.9067 - val_loss: 0.4992 - val_accuracy: 0.7956
Epoch 117/1000
7/7 [==============================] - 0s 47ms/step - loss: 0.4493 - accuracy: 0.8294 - val_loss: 0.4017 - val_accuracy: 0.8274
Epoch 118/1000
7/7 [==============================] - 0s 54ms/step - loss: 0.2351 - accuracy: 0.9067 - val_loss: 0.1525 - val_accuracy: 0.9464
Epoch 119/1000
7/7 [==============================] - 0s 47ms/step - loss: 0.1719 - accuracy: 0.9345 - val_loss: 0.4193 - val_accuracy: 0.8413
Epoch 120/1000
7/7 [==============================] - 0s 48ms/step - loss: 0.2868 - accuracy: 0.8909 - val_loss: 0.4756 - val_accuracy: 0.8155
Epoch 121/1000
7/7 [==============================] - 0s 53ms/step - loss: 0.5415 - accuracy: 0.8294 - val_loss: 0.1987 - val_accuracy: 0.9484
Epoch 122/1000
7/7 [==============================] - 0s 46ms/step - loss: 0.1646 - accuracy: 0.9464 - val_loss: 0.2404 - val_accuracy: 0.8988
Epoch 123/1000

7/7 [==============================] - 0s 48ms/step - loss: 0.0333 - accuracy: 0.9940 - val_loss: 0.0661 - val_accuracy: 0.9881
Epoch 174/1000
7/7 [==============================] - 0s 48ms/step - loss: 0.0322 - accuracy: 0.9921 - val_loss: 0.0591 - val_accuracy: 0.9841
Epoch 175/1000
7/7 [==============================] - 0s 48ms/step - loss: 0.0298 - accuracy: 0.9921 - val_loss: 0.0289 - val_accuracy: 0.9901
Epoch 176/1000
7/7 [==============================] - 0s 48ms/step - loss: 0.0285 - accuracy: 0.9901 - val_loss: 0.1120 - val_accuracy: 0.9563
Epoch 177/1000
7/7 [==============================] - 0s 47ms/step - loss: 0.9472 - accuracy: 0.8333 - val_loss: 0.0620 - val_accuracy: 0.9861
Epoch 178/1000
7/7 [==============================] - 0s 47ms/step - loss: 0.0512 - accuracy: 0.9901 - val_loss: 0.0873 - val_accuracy: 0.9782
Epoch 179/1000
7/7 [==============================] - 0s 47ms/step - loss: 0.0492 - accuracy: 0.9921 - val_loss: 0.0140 - val_accuracy: 0.9960
Epoch 180/1000

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

16/16 [==============================] - 0s 8ms/step
array([[38, 27, 31, 30],
       [34, 31, 25, 34],
       [36, 22, 24, 44],
       [33, 23, 33, 35]], dtype=int64)
              precision    recall  f1-score   support

           2       0.27      0.30      0.28       126
           3       0.30      0.25      0.27       124
           4       0.21      0.19      0.20       126
           5       0.24      0.28      0.26       124

    accuracy                           0.26       500
   macro avg       0.26      0.26      0.26       500
weighted avg       0.26      0.26      0.26       500

